In [8]:
from bs4 import BeautifulSoup
import smtplib
from email.MIMEMultipart import MIMEMultipart
from email.MIMEText import MIMEText
from email.MIMEBase import MIMEBase
from email import encoders
import os,sys,urllib2,json,pprint,requests,time,datetime
import numpy as np

In [147]:
def cricket_mail():
    url = "http://www.espncricinfo.com/ci/engine/match/index.html"
    data = BeautifulSoup(requests.get(url).text)
    search_link = data.findAll('a',href=True,text='Live scorecard')
    search_status = data.select('.match-status')
    search_tournament = data.select('.match-section-head')
    search_team1 = data.select('.innings-info-1')
    search_team2 = data.select('.innings-info-2')
    msg = '<html><body><table style="font-family:Segoe UI;border:1px solid black"><tr><td>TOURNAMENT</td><td>TEAM 1</td><td>TEAM 2</td><td>MATCH STATUS</td><td>LINK</td></tr>'
    for item,link in enumerate(search_link):
        try:
            match_status = search_status[item].findAll(text=True)[1].strip()
            team_1 = search_team1[item].findAll(text=True)[0].strip().replace('-',' ')
            team_2 = search_team2[item].findAll(text=True)[0].strip().replace('-',' ')
            tournament = link['href'].split('/')[-2].replace('-vs-','').replace('-',' ').replace(team_1,'').replace(team_2,'').strip().title()
            for tour in search_tournament:
                if tour.findAll(text=True)[0].strip().title() in tournament: tournament = tour.findAll(text=True)[0].strip().title()
            msg += '<tr><td>'+tournament+'</td><td>'+team_1+'</td><td>'+team_2+'</td><td>'+match_status+'</td><td><a href="'+link['href']+'">cricinfo</a></td></tr>'
        except Exception, e: print e
    msg += '</table></body></html>'
    return msg

In [150]:
def send_mail():
    fromaddr = 'souryapoddar290990@gmail.com'
    toaddr = ["souryapoddar290990@gmail.com"]
    password = "sourya1000"
    subject = "CRICKET UPDATE"
    body = cricket_mail()
    # print body
    msg = MIMEMultipart()
    msg['From'] = fromaddr
    msg['To'] = ",".join(toaddr)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'html'))
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(fromaddr,password)
    text = msg.as_string()
    server.sendmail(fromaddr, toaddr, text)
    server.quit()

send_mail()

In [ ]:
def send_push():
    from pushbullet import Pushbullet
    API_KEY = 'o.nYHrQiyqBr2NTj59HaQFSSGsgoLDYQrv'
    # API_KEY = 'o.gmWPEjdjJvbZRqnTvCc7sHkonggCW48I'
    pb = Pushbullet(API_KEY)
    url = "http://www.espncricinfo.com/ci/engine/match/index.html"
    data = BeautifulSoup(requests.get(url).text)
    search_list = data.findAll('a',href=True,text='Live scorecard')
    for item in search_list:
        link = "http://www.espncricinfo.com"+item['href']
        data = BeautifulSoup(requests.get(link).text).select('.match-information-strip')[0].findAll(text=True)[0].strip().split("\n")
        tournament = data[0].replace(",","")
        team_1 = BeautifulSoup(requests.get(link).text).select('.team-1-name')[0].findAll(text=True)[0].strip()
        team_2 = BeautifulSoup(requests.get(link).text).select('.team-2-name')[0].findAll(text=True)[0].strip()
        match_status = BeautifulSoup(requests.get(link).text).select('.innings-requirement')[0].findAll(text=True)[0].strip()
        title = "Cricket"
        text = team_1+" vs "+team_2+"\n"+match_status
        push = pb.push_note(title,text)

# send_push()